### Data Preprocessing

<p> First we will Load the data using data Loader </p>

In [1]:
%pwd

'd:\\MyCase\\Projects\\DSAI\\portfolio\\Sales_Forecasting_and_Analytics\\research'

In [2]:
import os 
os.chdir('../')
%pwd

'd:\\MyCase\\Projects\\DSAI\\portfolio\\Sales_Forecasting_and_Analytics'

<p> Loading the Data </p>

In [4]:
from pathlib import Path
from ml_service.components.data_ingestion import DataLoader
from ml_service.config.configuration import ConfigurationManager
from ml_service.constants import *

config_manager = ConfigurationManager(CONFIG_FILE_PATH)
data_acquisition_config = config_manager.get_data_acquisition_config()

loader = DataLoader(
    data_dir=Path(data_acquisition_config.local_dir),
    source=data_acquisition_config.source,
    data_files=data_acquisition_config.data_files,
    dataset_name=data_acquisition_config.dataset_name
)

loader.download()

[2025-11-29 17:43:15,408: INFO: main_utils: yaml file: config\config.yaml loaded successfully]
[2025-11-29 17:43:15,412: INFO: main_utils: yaml file: params.yaml loaded successfully]
[2025-11-29 17:43:15,414: INFO: main_utils: created directory at: artifacts\model]
[2025-11-29 17:43:15,416: INFO: main_utils: created directory at: artifacts/data_acquisition]
✅ All files already present in: artifacts\data_acquisition\store-sales-time-series-forecasting. Skipping download.


In [5]:
train_df = loader.load("train")
test_df = loader.load("test")
stores_df = loader.load("stores")
oil_df = loader.load("oil")
holidays_events_df = loader.load("holidays_events")
transactions_df = loader.load("transactions")

📥 Loading: artifacts\data_acquisition\store-sales-time-series-forecasting\train.csv
📥 Loading: artifacts\data_acquisition\store-sales-time-series-forecasting\test.csv
📥 Loading: artifacts\data_acquisition\store-sales-time-series-forecasting\stores.csv
📥 Loading: artifacts\data_acquisition\store-sales-time-series-forecasting\oil.csv
📥 Loading: artifacts\data_acquisition\store-sales-time-series-forecasting\holidays_events.csv
📥 Loading: artifacts\data_acquisition\store-sales-time-series-forecasting\transactions.csv


In [6]:
train_df.head()

,id,date,store_nbr,family,sales,onpromotion
0,0,2013-01-01,1,AUTOMOTIVE,0.0,0
1,1,2013-01-01,1,BABY CARE,0.0,0
2,2,2013-01-01,1,BEAUTY,0.0,0
3,3,2013-01-01,1,BEVERAGES,0.0,0
4,4,2013-01-01,1,BOOKS,0.0,0


In [7]:
test_df.head()

,id,date,store_nbr,family,onpromotion
0,3000888,2017-08-16,1,AUTOMOTIVE,0
1,3000889,2017-08-16,1,BABY CARE,0
2,3000890,2017-08-16,1,BEAUTY,2
3,3000891,2017-08-16,1,BEVERAGES,20
4,3000892,2017-08-16,1,BOOKS,0


In [8]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class DataPreprocessingConfig:
    """Config for preprocessing data."""
    root_dir: Path
    train_file: Path
    test_file: Path

In [9]:
from ml_service.constants import *
from ml_service.utils.main_utils import read_yaml, create_directories

In [10]:
class ConfigurationManager:
    def __init__(self, config_filepath: str):
        """Initialize the configuration manager.

        Args:
            config_filepath (str): Path to the main configuration file (YAML).
        """
        self.config = read_yaml(config_filepath)
        create_directories([self.config.artifacts_root])

    def get_data_preprocessing_config(self) -> DataPreprocessingConfig:
        """Get the configuration for data preprocessing.

        Returns:
            DataPreprocessingConfig: Paths for train and test preprocessed files.
        """
        config = self.config.data_preprocessing
        create_directories([config.root_dir])

        return DataPreprocessingConfig(
            root_dir=Path(config.root_dir),
            train_file=Path(config.root_dir) / config.train_file,
            test_file=Path(config.root_dir) / config.test_file
        )


<p>Let’s merge the train, stores, and transactions data (and similarly for the test set) for better clarity.</p>

In [11]:
train_merge_df = train_df.merge(stores_df, on='store_nbr', how='left')
train_merge_df = train_merge_df.merge(transactions_df, on=['store_nbr','date'], how='left')

test_merge_df = test_df.merge(stores_df, on='store_nbr', how='left')
test_merge_df = test_merge_df.merge(transactions_df, on=['store_nbr','date'], how='left')


In [12]:
train_merge_df.head()

,id,date,store_nbr,family,sales,onpromotion,city,state,type,cluster,transactions
0,0,2013-01-01,1,AUTOMOTIVE,0.0,0,Quito,Pichincha,D,13,NaN
1,1,2013-01-01,1,BABY CARE,0.0,0,Quito,Pichincha,D,13,NaN
2,2,2013-01-01,1,BEAUTY,0.0,0,Quito,Pichincha,D,13,NaN
3,3,2013-01-01,1,BEVERAGES,0.0,0,Quito,Pichincha,D,13,NaN
4,4,2013-01-01,1,BOOKS,0.0,0,Quito,Pichincha,D,13,NaN


<p>Now, Let’s merge the holidays_events and oil data (and similarly for the test set) for Date-level features.</p>

In [13]:
holidays_events_oil_merge_df = oil_df.merge(holidays_events_df, on='date', how='left')

In [14]:
holidays_events_oil_merge_df.head()

,date,dcoilwtico,type,locale,locale_name,description,transferred
0,2013-01-01,NaN,Holiday,National,Ecuador,Primer dia del ano,False
1,2013-01-02,93.14,NaN,NaN,NaN,NaN,NaN
2,2013-01-03,92.97,NaN,NaN,NaN,NaN,NaN
3,2013-01-04,93.12,NaN,NaN,NaN,NaN,NaN
4,2013-01-07,93.20,NaN,NaN,NaN,NaN,NaN


In [15]:
train_final = train_merge_df.merge(holidays_events_oil_merge_df, on='date', how='left')
test_final = test_merge_df.merge(holidays_events_oil_merge_df, on='date', how='left')

In [16]:
train_final.head()

,id,date,store_nbr,family,sales,onpromotion,city,state,type_x,cluster,transactions,dcoilwtico,type_y,locale,locale_name,description,transferred
0,0,2013-01-01,1,AUTOMOTIVE,0.0,0,Quito,Pichincha,D,13,NaN,NaN,Holiday,National,Ecuador,Primer dia del ano,False
1,1,2013-01-01,1,BABY CARE,0.0,0,Quito,Pichincha,D,13,NaN,NaN,Holiday,National,Ecuador,Primer dia del ano,False
2,2,2013-01-01,1,BEAUTY,0.0,0,Quito,Pichincha,D,13,NaN,NaN,Holiday,National,Ecuador,Primer dia del ano,False
3,3,2013-01-01,1,BEVERAGES,0.0,0,Quito,Pichincha,D,13,NaN,NaN,Holiday,National,Ecuador,Primer dia del ano,False
4,4,2013-01-01,1,BOOKS,0.0,0,Quito,Pichincha,D,13,NaN,NaN,Holiday,National,Ecuador,Primer dia del ano,False


In [17]:
train_final.isna().sum()

id                    0
date                  0
store_nbr             0
family                0
sales                 0
onpromotion           0
city                  0
state                 0
type_x                0
cluster               0
transactions     248358
dcoilwtico       933768
type_y          2680128
locale          2680128
locale_name     2680128
description     2680128
transferred     2680128
dtype: int64

###  Dropping Low‑Value Features

We drop the following columns:

* **`description`**, **`locale_name`**, **`locale`**, and **`transferred`**

#### ✅ Why?

* These fields contain textual or highly sparse information that **doesn’t directly affect sales prediction**.
* They introduce **noise** and **high cardinality**, making it harder for the model to learn meaningful patterns.
* Similar information (holidays) is already captured by the **`type`** column, making these redundant.

#### 💡 Result

By removing these columns, we:

* **Improve training efficiency** (smaller, cleaner dataset).
* Reduce the risk of overfitting.
* Maintain focus on the features that matter for forecasting sales.


In [18]:
train_final.drop(columns=['description', 'locale_name', 'locale', 'transferred'], inplace=True)
test_final.drop(columns=['description', 'locale_name', 'locale', 'transferred'], inplace=True)

In [19]:
train_final.head()

,id,date,store_nbr,family,sales,onpromotion,city,state,type_x,cluster,transactions,dcoilwtico,type_y
0,0,2013-01-01,1,AUTOMOTIVE,0.0,0,Quito,Pichincha,D,13,NaN,NaN,Holiday
1,1,2013-01-01,1,BABY CARE,0.0,0,Quito,Pichincha,D,13,NaN,NaN,Holiday
2,2,2013-01-01,1,BEAUTY,0.0,0,Quito,Pichincha,D,13,NaN,NaN,Holiday
3,3,2013-01-01,1,BEVERAGES,0.0,0,Quito,Pichincha,D,13,NaN,NaN,Holiday
4,4,2013-01-01,1,BOOKS,0.0,0,Quito,Pichincha,D,13,NaN,NaN,Holiday


In [20]:
train_final.shape

(3032964, 13)

In [21]:
train_final.isna().sum()

id                    0
date                  0
store_nbr             0
family                0
sales                 0
onpromotion           0
city                  0
state                 0
type_x                0
cluster               0
transactions     248358
dcoilwtico       933768
type_y          2680128
dtype: int64

<p> We impute transactions due to its high predictive potential, while dropping columns with excessive sparsity and low information value. </p>

In [22]:
train_final["dcoilwtico"] = train_final["dcoilwtico"].interpolate(method="linear")
train_final["dcoilwtico"] = train_final["dcoilwtico"].interpolate(method="linear")

In [23]:
train_final.isna().sum()

id                    0
date                  0
store_nbr             0
family                0
sales                 0
onpromotion           0
city                  0
state                 0
type_x                0
cluster               0
transactions     248358
dcoilwtico         1782
type_y          2680128
dtype: int64

In [24]:
train_final['dcoilwtico'] = train_final['dcoilwtico'].fillna(method='ffill')
test_final['dcoilwtico'] = test_final['dcoilwtico'].fillna(method='ffill')

C:\Users\rosha\AppData\Local\Temp\ipykernel_37288\1786350395.py:1: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  train_final['dcoilwtico'] = train_final['dcoilwtico'].fillna(method='ffill')
C:\Users\rosha\AppData\Local\Temp\ipykernel_37288\1786350395.py:2: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  test_final['dcoilwtico'] = test_final['dcoilwtico'].fillna(method='ffill')


In [25]:
train_final.isna().sum()

id                    0
date                  0
store_nbr             0
family                0
sales                 0
onpromotion           0
city                  0
state                 0
type_x                0
cluster               0
transactions     248358
dcoilwtico         1782
type_y          2680128
dtype: int64

### Data Cleaning

Note: Since transactions is the only column with missing values, we'll apply an imputation technique (such as filling with median or other suitable method) during the Data Transformation phase, where we'll also handle scaling, encoding, and splitting of the dataset. This ensures a clean and robust workflow.

<p>Now Let's look for duplicate values and handle them </p>

In [26]:
train_final.duplicated().sum()

np.int64(14256)

In [27]:
test_final.duplicated().sum()

np.int64(0)

In [28]:
train_final.drop_duplicates(inplace=True)

In [29]:
train_final.duplicated().sum()

np.int64(0)

<p>Since the data is clean  with no duplicate rows and no significant missing values (other than the <code>transactions</code> column, which we'll impute later) we can now move on to the next phase of the pipeline.</p>

In [30]:
from pathlib import Path
from typing import Dict
import pandas as pd


class DataProcessor:
    """Perform merging, cleaning, and exporting of Store Sales Time Series data."""

    def __init__(self, data_dir: Path, data_files: Dict[str, str]) -> None:
        self.data_dir = data_dir
        self.data_files = data_files
        self.data = {}

    def load(self) -> "DataProcessor":
        """Load all files and standardize date columns."""
        self.data = {
            name: pd.read_csv(self.data_dir / path) for name, path in self.data_files.items()
        }
        # ✅ Ensure 'date' columns are ALWAYS datetime
        for key in ["train", "test", "oil", "holidays_events", "transactions"]:
            if key in self.data and "date" in self.data[key].columns:
                self.data[key]["date"] = pd.to_datetime(self.data[key]["date"])
        return self

    def interpolate_oil(self) -> "DataProcessor":
        """Interpolate missing values in the oil data."""
        self.data["oil"].set_index("date", inplace=True)
        self.data["oil"]["dcoilwtico"] = self.data["oil"]["dcoilwtico"].interpolate(method="linear")
        self.data["oil"].reset_index(inplace=True)
        return self

    def merge_train_test(self) -> "DataProcessor":
        """Merge train/test with stores and transactions."""
        self.data["train_merged"] = (
            self.data["train"]
            .merge(self.data["stores"], on="store_nbr", how="left")
            .merge(self.data["transactions"], on=["store_nbr", "date"], how="left")
        )
        self.data["test_merged"] = (
            self.data["test"]
            .merge(self.data["stores"], on="store_nbr", how="left")
            .merge(self.data["transactions"], on=["store_nbr", "date"], how="left")
        )
        return self

    def merge_holidays_and_oil(self) -> "DataProcessor":
        """Merge holidays and oil data for final train/test."""
        holidays_oil_merged = self.data["oil"].merge(self.data["holidays_events"], on="date", how="left")

        self.data["train_final"] = self.data["train_merged"].merge(holidays_oil_merged, on="date", how="left")
        self.data["test_final"] = self.data["test_merged"].merge(holidays_oil_merged, on="date", how="left")

        # Final interpolate + forward fill
        for df_name in ["train_final", "test_final"]:
            self.data[df_name]["dcoilwtico"] = (
                self.data[df_name]["dcoilwtico"].interpolate(method="linear").ffill()
            )
        return self

    def drop_irrelevant_columns(self) -> "DataProcessor":
        """Drop low-value and sparse columns, remove duplicates."""
        drop_cols = ["description", "locale_name", "locale", "transferred"]
        for df_name in ["train_final", "test_final"]:
            self.data[df_name].drop(columns=drop_cols, errors="ignore", inplace=True)
            self.data[df_name].drop_duplicates(inplace=True)
        return self

    def save(self, train_file: Path, test_file: Path) -> "DataProcessor":
        """Save final merged train and test files."""
        train_file.parent.mkdir(parents=True, exist_ok=True) 
        self.data["train_final"].to_csv(train_file, index=False)
        self.data["test_final"].to_csv(test_file, index=False)

        print(f"✅ Final train saved to: {train_file}")
        print(f"✅ Final test saved to: {test_file}")

        return self


In [31]:
from pathlib import Path
from ml_service.config.configuration import ConfigurationManager
from ml_service.constants import CONFIG_FILE_PATH

config_manager = ConfigurationManager(CONFIG_FILE_PATH)
data_acquisition_config = config_manager.get_data_acquisition_config()
data_preprocessing_config = config_manager.get_data_preprocessing_config()

data_dir = Path(data_acquisition_config.local_dir)
files = data_acquisition_config.data_files

DataProcessor(data_dir, files) \
    .load() \
    .interpolate_oil() \
    .merge_train_test() \
    .merge_holidays_and_oil() \
    .drop_irrelevant_columns() \
    .save(data_preprocessing_config.train_file, data_preprocessing_config.test_file)


[2025-11-29 17:43:56,419: INFO: main_utils: yaml file: config\config.yaml loaded successfully]
[2025-11-29 17:43:56,422: INFO: main_utils: yaml file: params.yaml loaded successfully]
[2025-11-29 17:43:56,424: INFO: main_utils: created directory at: artifacts\model]
[2025-11-29 17:43:56,425: INFO: main_utils: created directory at: artifacts/data_acquisition]
[2025-11-29 17:43:56,426: INFO: main_utils: created directory at: artifacts/data_preprocessing]
✅ Final train saved to: artifacts\data_preprocessing\train_merged.csv
✅ Final test saved to: artifacts\data_preprocessing\test_merged.csv
